In [524]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('Annotations.csv')
df.drop(['url', 'URL(*)', 'Q2.11 Other media', 'Q2.12 News presented type', 'Q2.13 Visual bias'], axis=1, inplace=True)
df.fillna('', inplace=True)

# Rinomina le colonne utilizzando solo i numeri
df = df.rename(columns={
    'Q.1.1 Type of news': 'Q1.1',
    'Q.1.2 Accuracy': 'Q1.2',
    'Q.1.3 Headline': 'Q1.3',
    'Q1.3.1 Quotation': 'Q1.3.1',
    'Q.1.4 Fact-based lead': 'Q1.4',
    'Q.1.5 Bias': 'Q1.5',
    'Q.1.6 Sensational language': 'Q1.6',
    'Q1.7 Negative target': 'Q1.7',
    'Q1.7.1 If the article negatively targets a specific group, please check the types of targeting from the options below': 'Q1.7.1',
    'Q1.7.1.2 Please specify the type': 'Q1.7.1.2',
    'Q2.8 Attribution': 'Q2.8',
    'Q2.9.1 Date': 'Q2.9.1',
    'Q2.9.2 Exact Date': 'Q2.9.2',
    'Q2.10 event 30 days': 'Q2.10'
})

df


/tmp/ipykernel_11084/3864060508.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('', inplace=True)


,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1.0,2.0,3,,2.0,4.0,4.0,1.0,,,1.0,2.0,31-05-2021,2.0
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Marinella,,,,,,,,,,,,,,
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1.0,3.0,1;3,1.0,1.0,4.0,4.0,1.0,,,1.0,2.0,31-05-2021,2.0
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1.0,4.0,1;3,,2.0,4.0,4.0,1.0,,,3.0,2.0,31-05-2021,2.0
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1.0,4.0,1;3,,1.0,4.0,4.0,2.0,11,,3.0,2.0,31-05-2021,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3.0,3.0,1;8,,1.0,4.0,4.0,1.0,,,1.0,2.0,17-06-2021,2.0
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3.0,3.0,1;9,,2.0,4.0,4.0,1.0,,,1.0,2.0,17-06-2021,2.0
1060,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Manuel,,,,,,,,,,,,,,
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1.0,4.0,1;9,,1.0,4.0,3.0,1.0,,,3.0,2.0,25-06-2021,2.0


In [525]:
##########################################################################################################################################
##################################### CONTROLLO CHE NON CI SIANO VALORI STRANI ###########################################################
##########################################################################################################################################

def Q1_1(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '5', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_2(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_3(df, column):
    # Funzione per controllare se un valore è conforme
    def is_conform(value):
        return (value == '') or (all(1 <= int(x) <= 9 for x in value.split(';')) and all(re.match(r'^\d+$', x) for x in value.split(';')))
    
    # Trova tutti i valori non conformi
    non_conform_values = df[df[column].apply(lambda x: not is_conform(x))][column]
    
    # Stampa i valori non conformi
    primavolta = 0
    for value in non_conform_values:
        if value != (''):
            if (primavolta == 0):
                print("Valori non conformi in {}:".format(column))
            primavolta = 1
            print(value)

def Q1_31(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')

    valid_values = ['1', '2', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_4(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_5(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_6(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '4', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_7(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q1_71(df, column):
    # Funzione per controllare se un valore è conforme
    def is_conform(value):
        return (value == '') or (all(0 <= int(x) <= 12 for x in value.split(';') if x.isdigit()))

    # Trova tutti i valori non conformi
    non_conform_values = df[df[column].apply(lambda x: not is_conform(x))][column]

    # Stampa i valori non conformi
    primavolta = 0
    for value in non_conform_values:
        if value != (''):
            if (primavolta == 0):
                print("Valori non conformi in {}:".format(column))
            primavolta = 1
            print(value)

            
def Q2_8(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '3', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q2_91(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

def Q2_92(df, column):
    df[column] = df[column].astype(str)
    df[column] = pd.to_datetime(df[column], format='%d-%m-%Y')

def Q2_10(df, column):
    df[column] = df[column].astype(str)
    df[column] = df[column].str.replace('.0', '')
    valid_values = ['1', '2', '0', '']
    df[column] = df[column].apply(lambda x: x if x in valid_values else print(str(column) +' \"'+  str(x)+'\"'))

Q1_1(df, 'Q1.1')

Q1_2(df, 'Q1.2')

Q1_3(df, 'Q1.3')

Q1_31(df, 'Q1.3.1')

Q1_4(df, 'Q1.4')

Q1_5(df, 'Q1.5')

Q1_6(df, 'Q1.6')

Q1_7(df, 'Q1.7')

Q1_71(df, 'Q1.7.1')

Q2_8(df, 'Q2.8')

Q2_91(df, 'Q2.9.1')

Q2_92(df, 'Q2.9.2')

Q2_10(df, 'Q2.10')

# Sostituisci le celle vuote con NaN e poi ci metto -1
df.replace("", float("NaN"), inplace=True)
df = df.fillna(-1)


In [526]:
##########################################################################################################################################
##################################### CONTROLLO I SINGOLI VALORI DISTINTI PER COLONNA ####################################################
##########################################################################################################################################

saltaLePrimeQuattro = 1
for col in df.columns:
  saltaLePrimeQuattro = saltaLePrimeQuattro + 1
  if (saltaLePrimeQuattro > 4):
    print(f"Colonna {col}:")
    print(df[col].unique())

Colonna annotator:
['Manuel' 'Marinella' 'Angelo' 'marinella' -1]
Colonna Q1.1:
['1' -1 '2' '4' '3' '5']
Colonna Q1.2:
['2' -1 '3' '4' '1']
Colonna Q1.3:
['3' -1 '1;3' '3;9' '1;3;5' '1;3;4' '1;4' '4;9' '3;4' '1' '9' '1;3;9'
 '1;9' '1;5' '1;2;4' '1;2;3;4' '1;3;8' '1;8' '1;3;4;9' '1;2;3;4;9'
 '1;3;4;7;9' '3;4;9' '1;2' '1;3;5;9' '3;4;8' '7' '2;3' '1;5;9' '4;8' '8'
 '3;8' '3;6' '4' '5' '2' '1;2;9' '1;4;8;9' '1;3;4;5;9' '1;2;3;4;5;9'
 '1;2;8;9' '1;3;8;9' '1;2;4;9' '1;4;9' '1;7;9' '1;8;9']
Colonna Q1.3.1:
[-1 '1' '2']
Colonna Q1.4:
['2' -1 '1']
Colonna Q1.5:
['4' -1 '3' '2' '1']
Colonna Q1.6:
['4' -1 '2' '3' '1']
Colonna Q1.7:
['1' -1 '2']
Colonna Q1.7.1:
[-1 '11' '3' '12' '0' '1' '6' '2;5' '5' '2' '11;12' '5;11' '2;12' '10'
 '6;7' '7;11' '5;7' '9' '5;12' '10;11' '2;3;4;12' '2;3;4']
Colonna Q1.7.1.2:
[-1 'Fashion craft industries system' 'Mass production'
 'Government of Morocco' 'Right-wing' 'AIFA' 'Italian' 'Italians' 'PD'
 'liberals' 'European Union' 'M5S' 'Giunta siciliana' 'Lega;Ostella

In [527]:
df

,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1,2,3,-1,2,4,4,1,-1,-1,1,2,2021-05-31 00:00:00,2
1,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Marinella,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1,3,1;3,1,1,4,4,1,-1,-1,1,2,2021-05-31 00:00:00,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1,4,1;3,-1,2,4,4,1,-1,-1,3,2,2021-05-31 00:00:00,2
4,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Marinella,1,4,1;3,-1,1,4,4,2,11,-1,3,2,2021-05-31 00:00:00,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3,3,1;8,-1,1,4,4,1,-1,-1,1,2,2021-06-17 00:00:00,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3,3,1;9,-1,2,4,4,1,-1,-1,1,2,2021-06-17 00:00:00,2
1060,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Manuel,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1,4,1;9,-1,1,4,3,1,-1,-1,3,2,2021-06-25 00:00:00,2


In [528]:
df.dtypes


id           object
title        object
label        object
annotator    object
Q1.1         object
Q1.2         object
Q1.3         object
Q1.3.1       object
Q1.4         object
Q1.5         object
Q1.6         object
Q1.7         object
Q1.7.1       object
Q1.7.1.2     object
Q2.8         object
Q2.9.1       object
Q2.9.2       object
Q2.10        object
dtype: object

In [529]:
##########################################################################################################################################
##########################################################################################################################################
################## VERIFICO CHE TUTTI I DATI OBBLIGATORI SIANO STATI INSERITI, ALTRIMENTI TRONCO LA RIGA #################################
##########################################################################################################################################
##########################################################################################################################################


removeEmpty = (
    (df['Q1.1'] == -1) &
    (df['Q1.2'] == -1) &
    (df['Q1.3'] == -1) &
    (df['Q1.3.1'] == -1) &
    (df['Q1.4'] == -1) &
    (df['Q1.5'] == -1) &
    (df['Q1.6'] == -1) &
    (df['Q1.7'] == -1) &
    (df['Q1.7.1'] == -1) &
    (df['Q1.7.1.2'] == -1) &
    (df['Q2.8'] == -1) &
    (df['Q2.9.1'] == -1) &
    (df['Q2.9.2'] == -1) &
    (df['Q2.10'] == -1)
)

rules = (
    (df['Q1.1'] == -1) |
    (df['Q1.2'] == -1) |
    (df['Q1.4'] == -1) |
    (df['Q1.6'] == -1) |
    (df['Q1.7'] == -1) |
    ((df['Q1.7'] == 2) & (df['Q1.7.1'] == -1)) |
    ((df['Q1.7.1'] != -1) & (df['Q1.7.1.2'] == -1)) |
    (df['Q2.8'] == -1) |
    (df['Q2.9.1'] == -1) |
    ((df['Q2.9.2'] != '') & (df['Q2.9.1'] == -1)) |
    (df['Q2.10'] == -1)
)


result = df.loc[rules]

toRemove = df.loc[removeEmpty]


# Rimuovi i duplicati da toRemove
toRemove = toRemove.drop_duplicates()

# Trova le righe in toRemove che non sono già in result
new_rows = toRemove[~toRemove.index.isin(result.index)]

# Concatena le righe non duplicate di toRemove a result
result = pd.concat([result, new_rows])



df = df.drop(result.index, axis=0)


df

,id,title,label,annotator,Q1.1,Q1.2,Q1.3,Q1.3.1,Q1.4,Q1.5,Q1.6,Q1.7,Q1.7.1,Q1.7.1.2,Q2.8,Q2.9.1,Q2.9.2,Q2.10
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1,2,3,-1,2,4,4,1,-1,-1,1,2,2021-05-31 00:00:00,2
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1,3,1;3,1,1,4,4,1,-1,-1,1,2,2021-05-31 00:00:00,2
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1,4,1;3,-1,2,4,4,1,-1,-1,3,2,2021-05-31 00:00:00,2
6,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Group 1,Manuel,2,3,3,-1,1,3,2,2,11,Fashion craft industries system,3,1,-1,1
7,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Group 1,Marinella,2,3,3;9,-1,1,3,2,2,11,Mass production,3,1,-1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,ec3a0585-9da6-4732-bf58-c6e659d8e58e,La grande ondata di caldo nel Nord America,Group 1,Angelo,1,3,1;9,1,2,3,4,1,-1,-1,1,2,2021-06-30 00:00:00,2
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3,3,1;8,-1,1,4,4,1,-1,-1,1,2,2021-06-17 00:00:00,2
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3,3,1;9,-1,2,4,4,1,-1,-1,1,2,2021-06-17 00:00:00,2
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1,4,1;9,-1,1,4,3,1,-1,-1,3,2,2021-06-25 00:00:00,2


In [530]:
##########################################################################################################################################
####################################################### CASTO I TIPI E PULISCO ###########################################################
##########################################################################################################################################

####
# Converto le colonne numeriche in INT
####
columns_to_convert = ['Q1.1', 'Q1.2', 'Q1.3.1', 'Q1.4', 'Q1.5', 'Q1.6', 'Q1.7', 'Q2.8', 'Q2.9.1', 'Q2.10']
# Sotituisco i NaN in -1
df[columns_to_convert] = df[columns_to_convert].fillna(-1)
# Applico il cast
for col in columns_to_convert:
    df[columns_to_convert] = df[columns_to_convert].astype(int)


####################################################################################
# Faccio one hot delle colonne a valore multuiplo (Q1.3, 1.7.1)
####################################################################################
    
# Creiamo una nuova colonna per ciascun valore unico trovato nella colonna "Q1.3"
listQ_1_3 = [1,2,3,4,5,6,7,8,9]
for value in listQ_1_3:
    df[f'Q1.3_{value}'] = 0


# Creiamo una nuova colonna per ciascun valore unico trovato nella colonna "Q1.7"
listQ1_7_1 = [0,1,2,3,4,5,6,7,8,9,10,11,12]
for value in listQ1_7_1:
    df[f'Q1.7.1_{value}'] = 0

# Assegniamo 1 a ciascuna colonna appena creata se il valore corrisponde a quello nella colonna "Q1.3"
for idx, row in df.iterrows():
    values = str(row['Q1.3']).split(';')  # Convertiamo in stringa prima di fare lo split
    for value in values:
        df.at[idx, f'Q1.3_{value}'] = 1


# Assegniamo 1 a ciascuna colonna appena creata se il valore corrisponde a quello nella colonna "Q1.7.1"
for idx, row in df.iterrows():
    values = str(row['Q1.7.1']).split(';')  # Convertiamo in stringa prima di fare lo split
    for value in values:
        df.at[idx, f'Q1.7.1_{value}'] = 1


#elimino le colonne ridondanti
df.drop(['Q1.3','Q1.7.1','Q1.3_-1','Q1.7.1_-1'], axis=1, inplace=True)


# Visualizziamo il risultato
df

,id,title,label,annotator,Q1.1,Q1.2,Q1.3.1,Q1.4,Q1.5,Q1.6,...,Q1.7.1_3,Q1.7.1_4,Q1.7.1_5,Q1.7.1_6,Q1.7.1_7,Q1.7.1_8,Q1.7.1_9,Q1.7.1_10,Q1.7.1_11,Q1.7.1_12
0,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Manuel,1,2,-1,2,4,4,...,0,0,0,0,0,0,0,0,0,0
2,18d71db7-f362-46fa-953e-d8353c7f7232,Coronavirus| nuove regole: al tavolo al ristor...,Group 2,Angelo,1,3,1,1,4,4,...,0,0,0,0,0,0,0,0,0,0
3,3ca70f03-38f3-4a7b-bdd3-f012b22e0c7e,Israele: Lapid| ostacoli ma fiducioso su nuovo...,Group 2,Manuel,1,4,-1,2,4,4,...,0,0,0,0,0,0,0,0,0,0
6,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Group 1,Manuel,2,3,-1,1,3,2,...,0,0,0,0,0,0,0,0,1,0
7,34b92f4c-32db-454c-a369-ca2032443f6c,Le borsette fatte a 3 euro l'ora vendute a 820...,Group 1,Marinella,2,3,-1,1,3,2,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1056,ec3a0585-9da6-4732-bf58-c6e659d8e58e,La grande ondata di caldo nel Nord America,Group 1,Angelo,1,3,1,2,3,4,...,0,0,0,0,0,0,0,0,0,0
1058,5b30e244-4aa2-4ad3-b24a-2022c1415df7,Cosa fare dei documenti della Stasi,Group 1,Marinella,3,3,-1,1,4,4,...,0,0,0,0,0,0,0,0,0,0
1059,5b30e244-4aa2-4ad3-b24a-2022c1415df8,Cosa fare dei documenti della Stasi,Group 1,Angelo,3,3,-1,2,4,4,...,0,0,0,0,0,0,0,0,0,0
1061,eab45a62-1d47-4216-a51d-e45ccbcc3810,"Strani dolori allo stomaco, dopo 6 mesi muore ...",Group 1,Marinella,1,4,-1,1,4,3,...,0,0,0,0,0,0,0,0,0,0


In [531]:
df.dtypes


id           object
title        object
label        object
annotator    object
Q1.1          int64
Q1.2          int64
Q1.3.1        int64
Q1.4          int64
Q1.5          int64
Q1.6          int64
Q1.7          int64
Q1.7.1.2     object
Q2.8          int64
Q2.9.1        int64
Q2.9.2       object
Q2.10         int64
Q1.3_1        int64
Q1.3_2        int64
Q1.3_3        int64
Q1.3_4        int64
Q1.3_5        int64
Q1.3_6        int64
Q1.3_7        int64
Q1.3_8        int64
Q1.3_9        int64
Q1.7.1_0      int64
Q1.7.1_1      int64
Q1.7.1_2      int64
Q1.7.1_3      int64
Q1.7.1_4      int64
Q1.7.1_5      int64
Q1.7.1_6      int64
Q1.7.1_7      int64
Q1.7.1_8      int64
Q1.7.1_9      int64
Q1.7.1_10     int64
Q1.7.1_11     int64
Q1.7.1_12     int64
dtype: object

In [532]:
df.to_csv('Annotations_checked.csv', encoding='utf-8', index=False)